In [1]:
import os
from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [3]:
load_dotenv()

True

In [5]:
Groq_API_key = os.getenv("Groq_API_key")

In [7]:
embeddings = HuggingFaceEmbeddings()

C:\Users\ozt00103\AppData\Local\Temp\ipykernel_16688\3655315981.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
C:\Users\ozt00103\AppData\Local\Temp\ipykernel_16688\3655315981.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
D:\Buddhima\New_folder\Snapdrum\envs\llm_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stab

In [9]:
persistance_directly = 'movie_db'

vectordb = Chroma(
    embedding_function = embeddings,
    persist_directory = persistance_directly
)

In [11]:
retriever  = vectordb.as_retriever()

In [13]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

In [25]:
prompt = ChatPromptTemplate.from_template("""
Answer the question based on the following context:

Context: {context}

Question: {question}

Provide your response in the following JSON format:
{{
  "answer": "Your complete answer to the question",
  "contexts": ["Relevant excerpt from context 1", "Relevant excerpt from context 2"],
  "reasoning": "Explanation of how you used the context to answer the question"
}}

Important: 
- Return ONLY valid JSON, no additional text or markdown formatting.
- In the contexts array, include the most relevant excerpts from the provided context.
- Make the reasoning clear about which parts of the context were used.
""")

In [27]:
qa_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [29]:
answer = qa_chain.invoke("What is the plot of Kansas Saloon Smashers?")
print(answer)

{
  "answer": "The plot of Kansas Saloon Smashers involves a bartender serving drinks at a saloon. Carrie Nation and her followers burst in, assault a customer, and begin wrecking the bar. The bartender sprays seltzer water in Nation's face before policemen arrive and order everyone to leave.",
  "contexts": [
    "Book Name: Kansas Saloon Smashers | Description: A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside.",
    "They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register."
  ],
  "reasoning": "To answer the question, I used the context from the Document with id '0_0' which is specifically about 'Kansas Saloon Smashers'. I extracted the relevant parts of the page_content that describe the plot of the story, including th